In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
store_100_path = r"df_all_store_100.parquet"
store_42_path = r"df_all_store_42.parquet"
store_4906_path = r"df_all_store_4906.parquet"

In [3]:
df_100 = pd.read_parquet(store_100_path)
df_42 = pd.read_parquet(store_42_path)
df_4906 = pd.read_parquet(store_4906_path)
df_all = pd.concat([df_100, df_42, df_4906])

In [3]:
del df_100, df_42, df_4906

In [4]:
df_all['date'] = pd.to_datetime(df_all['date'])

In [5]:
warehouse_path = r"df_all_store_VZ01.parquet"
df_warehouse = pd.read_parquet(warehouse_path)

In [6]:
df_warehouse = df_warehouse.rename(columns={"stock": "warehouse_stock"})
df_warehouse['date'] = pd.to_datetime(df_warehouse['date'])

In [7]:
df_warehouse['sku'] = df_warehouse['sku'].astype(str)


In [8]:
df_warehouse = df_warehouse[df_warehouse['date'] > '2018-01-01']

In [9]:
df_all = df_all.merge(df_warehouse[["sku","date","warehouse_stock"]], on = ["sku","date"], how = "left")


In [10]:
df_all = df_all[df_all['date'] > '2018-01-01']

In [11]:
df_all = df_all.sort_values('date')

In [12]:
df_all = df_all.merge(df_all[df_all['stock'] > 0].groupby(['sku', 'store']).agg({'stock': 'first'}).reset_index().rename(columns={'stock':'palmers_first_allocation'}), on = ['sku', 'store'], how = 'left')

In [13]:
sku_store_df_step_g = df_all.groupby(['sku', 'store'])

In [14]:
i = 0
for store in ['42', '100']:
    for sku in ['100009161000001']:
        sku_store_df_step = sku_store_df_step_g.get_group((sku, store))
        first_stock = sku_store_df_step[sku_store_df_step["stock"]>0]["stock"].iloc[0]
        # nan in our_stock if before first stock
        sku_store_df_step.loc[sku_store_df_step["date"]<sku_store_df_step[sku_store_df_step["stock"]>0]["date"].iloc[0], "palmers_first_allocation"] = np.nan
        sku_store_df_step['days_from_first_stock'] = 1
        sku_store_df_step.loc[sku_store_df_step["date"]<sku_store_df_step[sku_store_df_step["stock"]>0]["date"].iloc[0], "days_from_first_stock"] = np.nan
        sku_store_df_step['days_from_first_stock'] = sku_store_df_step['days_from_first_stock'].cumsum()
        sku_store_df_step['sales_cumsum'] = sku_store_df_step['sales'].cumsum()
        sku_store_df_step['dynamic_stock_with_palmers_init'] = sku_store_df_step['palmers_first_allocation'] - sku_store_df_step['sales_cumsum']
        sku_store_df_step['sales_cumsum_divide_days_from_first_stock'] =  sku_store_df_step['sales_cumsum'] / sku_store_df_step['days_from_first_stock']
        zero_sales_days_back = 1
        sku_store_df_step['days_from_last_sales_that_not_zero'] = 1

        for index, row in sku_store_df_step.shift(1).iterrows():
            # if index is sku_store_df_step[sku_store_df_step["stock"]>0]["stock"] index
            if index > sku_store_df_step[sku_store_df_step["stock"]>0].index[0]:
                if row['sales'] == 0:
                    zero_sales_days_back += 1
                else:
                    zero_sales_days_back = 1

                sku_store_df_step.at[index, 'days_from_last_sales_that_not_zero'] = zero_sales_days_back
        sku_store_df_step.loc[sku_store_df_step["date"]<sku_store_df_step[sku_store_df_step["stock"]>0]["date"].iloc[0], "days_from_last_sales_that_not_zero"] = np.nan
        sku_store_df_step['rate_sales'] = sku_store_df_step['sales'] / sku_store_df_step['days_from_last_sales_that_not_zero']

In [19]:
print(sku_store_df_step.iloc[0:30])

In [21]:
sku_store_df_step.loc[sku_store_df_step['date'] < sku_store_df_step[sku_store_df_step["stock"]>0]['date'].iloc[0], "sum_of_our_allocation_stock"] = np.nan

{'3': 0.00888730705534243,
 '4': 0.00633550601965005,
 '5': 0.02311986898215578,
 '7': 0.016161844336196832,
 '8': 0.01692620264951282,
 '10': 0.008355408229065841,
 '11': 0.02208890459048387,
 '15': 0.006254079532664498,
 '18': 0.006992171237275469,
 '21': 0.008422388081263634,
 '22': 0.010594636621168522,
 '26': 0.005104915401820014,
 '27': 0.008597061029151996,
 '28': 0.002366621444322013,
 '29': 0.008356721559501092,
 '35': 0.004181644105838673,
 '36': 0.0030797598706632187,
 '37': 0.010212457464510528,
 '42': 0.01849169252833182,
 '43': 0.004591403201636935,
 '44': 0.00993403141223735,
 '45': 0.006219932941347976,
 '46': 0.010549983386369994,
 '47': 0.00498934232351794,
 '50': 0.005133808671395532,
 '51': 0.010039097847057418,
 '52': 0.00542799468889172,
 '55': 0.005265141714920616,
 '56': 0.004621609801647704,
 '57': 0.00697641127205246,
 '61': 0.003771885010040411,
 '63': 0.011553367838901636,
 '64': 0.004253877279777469,
 '67': 0.022115171199188887,
 '68': 0.012145679865199764,

In [28]:
sku_store_df_step.loc[sku_store_df_step["date"]<sku_store_df_step[sku_store_df_step["stock"]>0]]

In [29]:
print(df_all[(df_all['sku'] == '100009161000001') & (df_all['store'] == '100')])